In [17]:
import pickle

# Replace 'your_file.pkl' with the path to your .pkl file
filename = "../preds/jigsaw_base_p56_336_in1k_e10/train.pkl"

# Load the variable from the .pkl file
with open(filename, "rb") as f:
    combined_list = pickle.load(f)

In [18]:
len(combined_list)

2473

In [19]:
# Function to divide an image into patches
def divide_into_patches(image, patch_size):
    patches = []
    for i in range(0, image_size[0], patch_size[0]):
        for j in range(0, image_size[1], patch_size[1]):
            patch = image[i : i + patch_size[0], j : j + patch_size[1]]
            patches.append(patch)
    return patches


# Function to restore the image based on the permutation
def restore_image(patches, permutation):
    restored_image = np.zeros((image_size[0], image_size[1], 3), dtype=np.uint8)
    for index, permuted_index in enumerate(permutation):
        row = index // patches_per_row
        col = index % patches_per_row
        restored_image[
            row * patch_size[0] : (row + 1) * patch_size[0],
            col * patch_size[1] : (col + 1) * patch_size[1],
        ] = patches[permuted_index]
    return restored_image

In [ ]:
import os
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Image parameters
image_size = (336, 336)
patch_size = (56, 56)
patches_per_row = 6

# Path to the folder containing images
images_folder = Path("/workspace/data/study/cspuzzle/336/train")

# Get all image paths
image_paths = [str(p) for p in images_folder.rglob("*.jpg")]

# Sort the image paths (if needed)
image_paths = sorted(image_paths)

# Check if the number of images matches the length of combined_list
if len(image_paths) != len(combined_list):
    print("Error: The number of images and the length of combined_list do not match.")
else:
    for i, (image_path, combined_item) in enumerate(zip(image_paths, combined_list)):
        if i < 20:
            continue
        if i >= 40:  # Show only the first 5 images
            break

        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Failed to load image from {image_path}")
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Divide the image into patches
        patches = divide_into_patches(image, patch_size)

        # Get the permutation from combined_list
        permutation = combined_item[0]

        # Restore the image
        restored_image = restore_image(patches, permutation)

        # Show the images
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.title("Original Image")

        plt.subplot(1, 2, 2)
        plt.imshow(restored_image)
        plt.title("Restored Image")

        plt.show()
        print(
            f"Image: {image_path}, Combined Item: {combined_item[1]}, Permutation: {permutation}"
        )

In [1]:
import math

# Number of patches along one dimension
patches_per_dim = 3

# Total number of patches
total_patches = patches_per_dim**2

# Calculate number of possible cases for permutating one pair of patches
num_cases = math.comb(total_patches, 4)

print(num_cases)

126


In [29]:
import random
import numpy as np


def generate_permutations(n, length):
    permutations = set()
    while len(permutations) < n:
        permutation = random.sample(range(length), length)
        permutations.add(tuple(permutation))
    return permutations


permutations = generate_permutations(3, 36)
permutations = np.array(list(permutations), dtype=np.int32)
np.save("perm6x6x3.npy", permutations)

In [28]:
permutations[0]

array([19, 23, 26,  2,  5, 28,  1,  0, 29, 34, 35, 16, 21, 24,  8, 22, 30,
        3, 33, 11,  9,  6, 10, 20, 12, 13, 17, 15, 31, 27, 25,  4, 32, 18,
       14,  7], dtype=int32)

In [24]:
perm6x6x500 = np.load("perm6x6x500.npy")
perm6x6x1000 = np.load("perm6x6x1000.npy")

In [25]:
print(perm6x6x500.shape, type(perm6x6x500))
print(perm6x6x1000.shape, type(perm6x6x1000))

(500, 36) <class 'numpy.ndarray'>
(1000, 36) <class 'numpy.ndarray'>


In [1]:
import random
import numpy as np


def generate_permutations(n, length, k):
    permutations = set()
    base_sequence = list(range(length))

    while len(permutations) < n:
        indices_to_permute = random.sample(range(length), k)
        permuted_part = random.sample(indices_to_permute, k)

        permutation = base_sequence.copy()
        for i, index in enumerate(indices_to_permute):
            permutation[index] = base_sequence[permuted_part[i]]

        permutations.add(tuple(permutation))

    return permutations


n = 50
length = 36
k = 6
permutations = generate_permutations(n, length, k)
permutations = np.array(list(permutations), dtype=np.int32)
np.save("perm6x6x3.npy", permutations)
# permutations

array([[ 0,  1,  2, ..., 33,  3, 35],
       [ 0,  1,  2, ..., 33, 34, 35],
       [ 0,  1,  2, ..., 33, 34, 25],
       ...,
       [ 0,  1,  2, ..., 33, 34, 35],
       [ 0, 27,  2, ..., 33, 34, 35],
       [ 0,  1,  2, ..., 33, 34, 35]], dtype=int32)

In [1]:
import os
import shutil

# Define the source and destination directories
source_directory = "../outputs"
destination_directory = "../logs"

# Iterate through all subdirectories in the source directory
for subdir, _, files in os.walk(source_directory):
    if "log.txt" in files:
        # Construct the source file path
        source_file = os.path.join(subdir, "log.txt")

        # Construct the destination file path
        relative_subdir_path = os.path.relpath(subdir, source_directory)
        destination_subdir = os.path.join(destination_directory, relative_subdir_path)
        destination_file = os.path.join(destination_subdir, "log.txt")

        # Create the destination subdirectory if it does not exist
        os.makedirs(destination_subdir, exist_ok=True)

        # Copy the file
        shutil.copy2(source_file, destination_file)
        print(f"Copied: {source_file} -> {destination_file}")

Copied: ../outputs/in1k_jigsaw_small_patch56_336_e30_c500/log.txt -> ../logs/in1k_jigsaw_small_patch56_336_e30_c500/log.txt
Copied: ../outputs/jigsaw_b_p56_336_in1k_v2_m0_10e/log.txt -> ../logs/jigsaw_b_p56_336_in1k_v2_m0_10e/log.txt
Copied: ../outputs/jigsaw_b_p56_336_in1k_m3_20e/log.txt -> ../logs/jigsaw_b_p56_336_in1k_m3_20e/log.txt
Copied: ../outputs/in1k_jigsaw_small_patch56_336_e30_cls50/log.txt -> ../logs/in1k_jigsaw_small_patch56_336_e30_cls50/log.txt
Copied: ../outputs/in1k_jigsaw_tiny_patch56_336_e10_c50/log.txt -> ../logs/in1k_jigsaw_tiny_patch56_336_e10_c50/log.txt
Copied: ../outputs/in1k_jigsaw_base_patch56_336_e10_c500/log.txt -> ../logs/in1k_jigsaw_base_patch56_336_e10_c500/log.txt
Copied: ../outputs/in1k_jigsaw_base_patch56_336_e10_c1000/log.txt -> ../logs/in1k_jigsaw_base_patch56_336_e10_c1000/log.txt
Copied: ../outputs/jigsaw_base_p56_336_in1k/log.txt -> ../logs/jigsaw_base_p56_336_in1k/log.txt
Copied: ../outputs/in1k_jigsaw_base_patch56_336_e10_c50ftc1000/log.txt -> 